# Laboratorio 5 - Parte 1

### Máquinas de Vectores de Soporte

### 2018-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer Integrante: Deiry Sofía Navas Muriel
#### Segundo Integrante: David Alejandro Alzate Marín


In [13]:
from __future__ import division
import numpy as np
import math
import matplotlib.pyplot as plt
#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")
%matplotlib inline

## Ejercicio 1

En este ejercicio deben resolver un problema de clasificación multi-clase usando una SVM. Teniendo en cuenta que la formulación original de la SVM sólo permite resolver problemas bi-clase, deben comparar los resultados obtenidos usando una estrategia **Uno vs Uno** con una estrategia **Uno vs El resto**.

Cargamos los datos:

In [15]:
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
digits = load_digits(n_class=4)

#--------- preprocesamiento--------------------
pca = PCA(0.99, whiten=True)
data = pca.fit_transform(digits.data)

#---------- Datos a usar ----------------------
X = data
Y = digits.target
print(X.shape)
Nclases = np.unique(Y)
print(Nclases)
for i in Nclases:
    print("clase "+ str(i) + " tiene " + str(len(X[Y==i])))

(720, 39)
[0 1 2 3]
clase 0 tiene 178
clase 1 tiene 182
clase 2 tiene 177
clase 3 tiene 183


Consutar el manejo de la librería sklearn para entrenar un modelos SVM en: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC. Para el caso multiclase la librería ya tiene implementada la estrategia **Uno vs Uno**, así que en este caso sólo deben llamar correctamente los métodos. Complete el siguiente código usando un clasificador basado en SVM:

In [69]:
import math
import numpy as np
from numpy import random
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold

def mainSVM(kernel='linear', c=0.001 ,gamma=0):

    if gamma is 0:
        gamma = 'auto'
        
    #Validamos el modelo
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    percentVectors = np.zeros(Folds)
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        scaler = preprocessing.StandardScaler().fit(Xtrain)
        Xtrain = scaler.transform(Xtrain)
        Xtest = scaler.transform(Xtest)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        modelo = SVC(C= c,kernel=kernel,gamma=gamma,  decision_function_shape='ovo')
        modelo.fit(Xtrain, Ytrain)
        
        #Calculamos el porcentaje de vector de soporte
        percentVectors[j] = (len(modelo.support_vectors_)/len(Xtrain))

        #Validación
        Ytrain_pred = modelo.predict(Xtrain)
        Yest = modelo.predict(Xtest)

        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
        EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        j += 1

        
    #print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
    #print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
    #print('% Vectore de soporte = ', modelo.n_support_.sum()/720)
    mean = round(np.mean(EficienciaVal),5)
    std = round(np.std(EficienciaVal),5)
    percent = round(percentVectors.sum()/4, 5)
    return mean,std,percent

mean,std,percent = mainSVM(kernel='linear',c=1, gamma=0)

print (mean, std, percent)

0.97078 0.01549 0.26204


Realice los experimientos necesarios para llenar la siguiente tabla:

In [48]:
kernels = ['lineal','lineal','lineal','lineal','lineal','lineal','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']
cs = [0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]
gammas = [0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1]
medias = np.zeros(len(kernels))
stds = np.zeros(len(kernels))
percents = np.zeros(len(kernels))

for i in range(0,len(kernels)):
    if kernels[i] is 'lineal':
        kernel = 'linear'
    elif kernels[i] is 'rbf':
        kernel =  'rbf'
    medias[i],stds[i],percents[i] = mainSVM(kernel=kernel, c=cs[i], gamma=gammas[i])

0.796 0.0533 1.0
0.9707 0.02926 0.60832
0.98045 0.01661 0.2866
0.97078 0.01549 0.26204
0.97078 0.01549 0.26204
0.97078 0.01549 0.26204
0.31562 0.10806 1.0
0.27791 0.04275 1.0
0.25417 0.00172 0.99954
0.31562 0.10806 1.0
0.27791 0.04275 1.0
0.25417 0.00172 1.0
0.83203 0.03645 1.0
0.27791 0.04275 1.0
0.25417 0.00172 1.0
0.98466 0.01152 0.64305
0.80855 0.04071 0.97267
0.25696 0.00649 1.0
0.99025 0.00832 0.57732
0.81547 0.0377 0.97221
0.25837 0.00603 1.0
0.99025 0.00832 0.57732
0.81547 0.0377 0.97221
0.25837 0.00603 1.0


In [49]:
randn = np.random.randn
df_types = pd.DataFrame({
    'Numero de arboles' : pd.Series([5,5,5,5,5,5,10,10,10,10,10,10,20,20,20,20,20,20,50,50,50,50,50,50,100,100,100,100,100,100]), 'Variables analizadas por nodo' : pd.Series([5,10,15,20,25,30,5,10,15,20,25,30,5,10,15,20,25,30,5,10,15,20,25,30,5,10,15,20,25,30])})
df_types["Eficiencia en validacion"] = efi_val
df_types["Intervalo de confianza"] = interval
df_types.set_index(['Numero de arboles','Variables analizadas por nodo'], inplace=True)
#df_types.sort_index(inplace=True)
df_types["Eficiencia en validacion"][0]=0.8778
df_types["Intervalo de confianza"][0] = 0.0143
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [50]:
qgrid_widget.get_changed_df()

Eficiencia en validacion  Intervalo de confianza  \
Kernel C       gamma                                                     
lineal 0.001   0.00                    0.79600                 0.05330   
       0.010   0.00                    0.97070                 0.02926   
       0.100   0.00                    0.98045                 0.01661   
       1.000   0.00                    0.97077                 0.01548   
       10.000  0.00                    0.97078                 0.01549   
       100.000 0.00                    0.97078                 0.01549   
rbf    0.001   0.01                    0.31562                 0.10806   
               0.10                    0.27791                 0.04275   
               1.00                    0.25417                 0.00172   
       0.010   0.01                    0.31562                 0.10806   
               0.10                    0.27791                 0.04275   
               1.00                    0.25417                 0.00172   
       0.100   0.01                    0.83203                 0.03645   
               0.10                    0.27791                 0.04275   
               1.00                    0.25417                 0.00172   
       1.000   0.01                    0.98466                 0.01152   
               0.10                    0.80855                 0.04071   
               1.00                    0.25696                 0.00649   
       10.000  0.01                    0.99025                 0.00832   
               0.10                    0.81547                 0.03770   
               1.00                    0.25837                 0.00603   
       100.000 0.01                    0.99025                 0.00832   
               0.10                    0.81547                 0.03770   
               1.00                    0.25837                 0.00603   

                      % de Vectores de Soporte  
Kernel C       gamma                            
lineal 0.001   0.00                    1.00000  
       0.010   0.00                    0.60832  
       0.100   0.00                    0.28660  
       1.000   0.00                    0.26200  
       10.000  0.00                    0.26204  
       100.000 0.00                    0.26204  
rbf    0.001   0.01                    1.00000  
               0.10                    1.00000  
               1.00                    0.99954  
       0.010   0.01                    1.00000  
               0.10                    1.00000  
               1.00                    1.00000  
       0.100   0.01                    1.00000  
               0.10                    1.00000  
               1.00                    1.00000  
       1.000   0.01                    0.64305  
               0.10                    0.97267  
               1.00                    1.00000  
       10.000  0.01                    0.57732  
               0.10                    0.97221  
               1.00                    1.00000  
       100.000 0.01                    0.57732  
               0.10                    0.97221  
               1.00                    1.00000

## Ejercicio 2

Cree dos funciones, una para entrenar un conjunto de modelos bajo la estrategia Uno vs el resto, usando como clasificador base una SVM. La segunda función debe usar el conjunto de modelos entrenados, y clasificar un conjunto de muestras de validación.

In [70]:
from sklearn.multiclass import OneVsRestClassifier

def TrainSVM_OnevsRest(Xtrain, Ytrain, c=1, g=0,k='linear'):
        if g is 0:
            g = 'auto'
        models = []
        Y_new = np.copy(Ytrain)
        for i in Nclases:
            Y_new[Ytrain == i] = 1
            Y_new[Ytrain != i] = 0
            modelo = SVC(C=c,kernel=k,gamma=g, decision_function_shape='ovr',probability=True)
            modelo.fit(Xtrain, Y_new)   
            models.append(modelo)  
        return  models
       

def ValidaSVM_OnevsRest(X, models):   
    Y = np.zeros([X.shape[0],1])  
    prob = np.zeros([X.shape[0], len(models)])
    j=0    
    for model in models:           
        currentprob = model.predict_proba(X)        
        prob[:,j] = currentprob[:,1]
        j+=1
    Y = np.argmax(prob,axis=1)   
    return Y

## Ejercicio 3

Use las funciones definidas en el punto anterior para llevar a cabo la prueba de simulación con el mismo conjunto de datos del punto 1.

In [72]:
Folds = 4
random.seed(19680801)
EficienciaTrain = np.zeros(Folds)
EficienciaVal = np.zeros(Folds)
skf = StratifiedKFold(n_splits=Folds)
j = 0
for train, test in skf.split(X, Y):
    Xtrain = X[train,:]
    Ytrain = Y[train]
    Xtest = X[test,:]
    Ytest = Y[test]
      
    #Normalizamos los datos
    scaler = preprocessing.StandardScaler().fit(Xtrain)
    Xtrain = scaler.transform(Xtrain)
    Xtest = scaler.transform(Xtest)
    
    #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
    models = []
    models = TrainSVM_OnevsRest(Xtrain, Ytrain)
      
    #Validación
    Ytrain_pred = ValidaSVM_OnevsRest(Xtrain,models)
    Yest = ValidaSVM_OnevsRest(Xtest, models)
    
    #Evaluamos las predicciones del modelo con los datos de test
    EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
    EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
    j += 1
        
print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))

Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9777370884748747+-0.01637228483712346


Realice los experimientos necesarios para llenar la siguiente tabla:

In [35]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['lineal','lineal','lineal','lineal','lineal','lineal','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)
df_types["Eficiencia en validacion"][3] = "0.97633"
df_types["Intervalo de confianza"][3] = "0.01837"
df_types["% de Vectores de Soporte"][3] = "0.2778"
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

UWdyaWRXaWRnZXQoZ3JpZF9vcHRpb25zPXsnaGlnaGxpZ2h0U2VsZWN0ZWRSb3cnOiBUcnVlLCAnZnVsbFdpZHRoUm93cyc6IFRydWUsICdyb3dIZWlnaHQnOiAyOCwgJ2VuYWJsZUNvbHVtblLigKY=


Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [36]:
qgrid_widget.get_changed_df()

Eficiencia en validacion Intervalo de confianza  \
Kernel C       gamma                                                   
lineal 0.001   0.00                                                    
       0.010   0.00                                                    
       0.100   0.00                                                    
       1.000   0.00                   0.97633                0.01837   
       10.000  0.00                                                    
       100.000 0.00                                                    
rbf    0.001   0.01                                                    
               0.10                                                    
               1.00                                                    
       0.010   0.01                                                    
               0.10                                                    
               1.00                                                    
       0.100   0.01                                                    
               0.10                                                    
               1.00                                                    
       1.000   0.01                                                    
               0.10                                                    
               1.00                                                    
       10.000  0.01                                                    
               0.10                                                    
               1.00                                                    
       100.000 0.01                                                    
               0.10                                                    
               1.00                                                    

                     % de Vectores de Soporte  
Kernel C       gamma                           
lineal 0.001   0.00                            
       0.010   0.00                            
       0.100   0.00                            
       1.000   0.00                    0.2778  
       10.000  0.00                            
       100.000 0.00                            
rbf    0.001   0.01                            
               0.10                            
               1.00                            
       0.010   0.01                            
               0.10                            
               1.00                            
       0.100   0.01                            
               0.10                            
               1.00                            
       1.000   0.01                            
               0.10                            
               1.00                            
       10.000  0.01                            
               0.10                            
               1.00                            
       100.000 0.01                            
               0.10                            
               1.00

*En las tablas el punto separa las cifras decimales, es decir 100.000 es 100 ó 1.000 es 1.